In [1]:
!pip install sqlalchemy==1.3.9

#Please uncomment and execute the code below if you are working locally.
#!pip install ipython-sql


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.0 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=34aab375cb264d069d2ccc2c3768e2184cf35b4c9ed4757f94c534bed0190509
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


In [2]:
%load_ext sql
#%reload_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()



In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [25]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")
print(df.columns[:11])

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing_Outcome'],
      dtype='object')


In [10]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null


 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [58]:
# Tasks
# Now write and execute SQL queries to solve the assignment tasks.
# Note: If the column names are in mixed case enclose it in double quotes For Example "Landing_Outcome"
# Task 1
# Display the names of the unique launch sites in the space mission

res = cur.execute("SELECT DISTINCT Launch_Site FROM SPACEXTBL")
res.fetchone()

### Task 2
#####  Display 5 records where launch sites begin with the string 'CCA'
res = cur.execute("SELECT * FROM SPACEXTBL WHERE Launch_Site LIKE 'CCA%' LIMIT 5")
res.fetchone()

# Task 3
# Display the total payload mass carried by boosters launched by NASA (CRS)
res = cur.execute("SELECT SUM(PAYLOAD_MASS__KG_) AS TotalPayloadMass FROM SPACEXTBL WHERE Launch_Site = 'NASA CRS'")
res.fetchone()

# Task 4
# Display average payload mass carried by booster version F9 v1.1

res = cur.execute("SELECT AVG(PAYLOAD_MASS__KG_) AS AveragePayloadMass FROM SPACEXTBL WHERE Booster_Version = 'F9 v1.1'")
res.fetchone()

# Task 5
# List the date when the first succesful landing outcome in ground pad was acheived.
res = cur.execute("SELECT MIN(Date) AS FirstSuccessfulGroundPadLanding FROM SPACEXTBL WHERE Landing_Outcome = 'Success'")
#                  AND Landing_Pad IS NOT NULL")
res.fetchone()

#Task 6
# List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

res = cur.execute("SELECT Booster_Version FROM SPACEXTBL WHERE Landing_Outcome = 'Success' AND PAYLOAD_MASS__KG_ > 4000 AND PAYLOAD_MASS__KG_ < 6000")
res.fetchone()

# Task 7
# List the total number of successful and failure mission outcomes
res = cur.execute("SELECT Mission_Outcome, COUNT(*) AS Total FROM SPACEXTBL WHERE Mission_Outcome IN ('Success', 'Failure') GROUP BY Mission_Outcome")
res.fetchone()

# Task 8
# List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

res = cur.execute("""
        SELECT Booster_Version
        FROM SPACEXTBL
        WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL)
        """)
res.fetchone()

# Task 9
# List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
# Note: SQLLite does not support monthnames. So you need to use substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.

res = cur.execute("""
        SELECT 
            CASE
                WHEN substr(Date, 4, 2) = '01' THEN 'January'
                WHEN substr(Date, 4, 2) = '02' THEN 'February'
                WHEN substr(Date, 4, 2) = '03' THEN 'March'
                WHEN substr(Date, 4, 2) = '04' THEN 'April'
                WHEN substr(Date, 4, 2) = '05' THEN 'May'
                WHEN substr(Date, 4, 2) = '06' THEN 'June'
                WHEN substr(Date, 4, 2) = '07' THEN 'July'
                WHEN substr(Date, 4, 2) = '08' THEN 'August'
                WHEN substr(Date, 4, 2) = '09' THEN 'September'
                WHEN substr(Date, 4, 2) = '10' THEN 'October'
                WHEN substr(Date, 4, 2) = '11' THEN 'November'
                WHEN substr(Date, 4, 2) = '12' THEN 'December'
            END AS MonthName,
            Booster_Version,
            Launch_Site
        FROM 
            SPACEXTBL
        WHERE 
            substr(Date, 0, 5) = '2015' AND
            Landing_Outcome = 'Failure'
        """)
res.fetchone()

# Task 10
# Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

res = cur.execute("""
        SELECT  Landing_Outcome,
            COUNT(*) AS OutcomeCount
        FROM 
            SPACEXTBL
        WHERE 
            Date BETWEEN '2010-06-04' AND '2017-03-20'
        GROUP BY 
            Landing_Outcome
        ORDER BY 
            OutcomeCount DESC
        """)
res.fetchone()

('No attempt', 10)